#### lstm 二分类

In [1]:
import torch
from torch import nn

# 定义一个LSTM模型
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # 初始化隐藏状态h0, c0为全0向量
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # 将输入x和隐藏状态(h0, c0)传入LSTM网络
        out, _ = self.lstm(x, (h0, c0))
        # 取最后一个时间步的输出作为LSTM网络的输出
        out = self.fc(out[:, -1, :])
        return out

# 定义LSTM超参数
input_size = 10   # 输入特征维度
hidden_size = 32  # 隐藏单元数量
num_layers = 2    # LSTM层数
output_size = 2   # 输出类别数量

# 构建一个随机输入x和对应标签y
x = torch.randn(64, 5, 10)  # [batch_size, sequence_length, input_size]
y = torch.randint(0, 2, (64,))  # 二分类任务，标签为0或1

# 创建LSTM模型，并将输入x传入模型计算预测输出
lstm = LSTM(input_size, hidden_size, num_layers, output_size)
pred = lstm(x)  # [batch_size, output_size]

# 定义损失函数和优化器，并进行模型训练
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=1e-3)
num_epochs = 100

for epoch in range(num_epochs):
    # 前向传播计算损失函数值
    pred = lstm(x)  # 在每个epoch中重新计算预测输出
    loss = criterion(pred.squeeze(), y)

    # 反向传播更新模型参数
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 输出每个epoch的训练损失
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


/home/zhangmanman/anaconda3/envs/NLP_lab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch [1/100], Loss: 0.7134
Epoch [2/100], Loss: 0.7111
Epoch [3/100], Loss: 0.7088
Epoch [4/100], Loss: 0.7067
Epoch [5/100], Loss: 0.7046
Epoch [6/100], Loss: 0.7025
Epoch [7/100], Loss: 0.7005
Epoch [8/100], Loss: 0.6986
Epoch [9/100], Loss: 0.6967
Epoch [10/100], Loss: 0.6947
Epoch [11/100], Loss: 0.6928
Epoch [12/100], Loss: 0.6910
Epoch [13/100], Loss: 0.6891
Epoch [14/100], Loss: 0.6872
Epoch [15/100], Loss: 0.6852
Epoch [16/100], Loss: 0.6833
Epoch [17/100], Loss: 0.6813
Epoch [18/100], Loss: 0.6794
Epoch [19/100], Loss: 0.6773
Epoch [20/100], Loss: 0.6753
Epoch [21/100], Loss: 0.6733
Epoch [22/100], Loss: 0.6712
Epoch [23/100], Loss: 0.6691
Epoch [24/100], Loss: 0.6670
Epoch [25/100], Loss: 0.6649
Epoch [26/100], Loss: 0.6628
Epoch [27/100], Loss: 0.6607
Epoch [28/100], Loss: 0.6587
Epoch [29/100], Loss: 0.6567
Epoch [30/100], Loss: 0.6547
Epoch [31/100], Loss: 0.6528
Epoch [32/100], Loss: 0.6510
Epoch [33/100], Loss: 0.6492
Epoch [34/100], Loss: 0.6474
Epoch [35/100], Loss: 0

In [2]:
# ((pred>0)[:,:1].T[0]),y

In [14]:
torch.max(pred,1)

torch.return_types.max(
values=tensor([1.2742, 0.5629, 1.7146, 0.6699, 0.7747, 1.6594, 0.7094, 0.6375, 0.9172,
        0.6195, 0.7075, 0.3255, 1.6340, 0.1935, 1.7029, 1.9187, 0.4304, 1.7519,
        0.8964, 1.2886, 0.1921, 1.3157, 0.3414, 2.1218, 0.3145, 1.5517, 1.0473,
        0.6802, 1.1804, 0.7574, 2.0330, 0.2840, 1.6806, 0.1962, 1.6692, 0.4877,
        0.6007, 0.2638, 1.1992, 1.4083, 1.8598, 0.6409, 1.6314, 0.8195, 0.5100,
        0.8480, 0.5383, 1.6253, 0.6145, 0.3386, 1.2286, 0.6351, 0.7723, 0.6185,
        0.5660, 0.6822, 0.8704, 0.1046, 1.1036, 1.3511, 1.2697, 0.0896, 1.0423,
        1.9787], grad_fn=<MaxBackward0>),
indices=tensor([0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
        1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0]))

In [15]:
# Setup
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.datasets as dsets
import torchvision.transforms as transforms

torch.manual_seed(1)

# Hyper Parameters
epochs = 10           # 训练整批数据多少次, 为了节约时间, 我们只训练一次
batch_size = 64
time_step = 28      # rnn 时间步数 / 图片高度
input_size = 28     # rnn 每步输入值 / 图片每行像素
hidden_size = 64
num_layers = 1
num_classes = 10
lr = 0.01           # learning rate


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Mnist手写数字
train_data = dsets.MNIST(root='./mnist/',    # 保存或者提取位置
                         train=True,  # this is tra`ining data
                         transform=transforms.ToTensor(),    # 转换 PIL.Image or numpy.ndarray 成
                                                    # torch.FloatTensor (C x H x W), 训练的时候 normalize 成 [0.0, 1.0] 区间
                         download=True,          # 没下载就下载, 下载了就不用再下了改成False
)

test_data = dsets.MNIST(root='./mnist/',
                        train=False,
                        transform=transforms.ToTensor())
# batch_size = 64
# Dataloader
# PyTorch中数据读取的一个重要接口，该接口定义在dataloader.py中，只要是用PyTorch来训练模型基本都会用到该接口（除非用户重写…），
# 该接口的目的：将自定义的Dataset根据batch size大小、是否shuffle等封装成一个Batch Size大小的Tensor，用于后面的训练。
train_loader = DataLoader(dataset=train_data,
                          batch_size=batch_size,
                          shuffle=True) # 在每个epoch开始的时候，对数据重新打乱进行训练。在这里其实没啥用，因为只训练了一次

test_loader = DataLoader(dataset=test_data,
                         batch_size=batch_size,
                         shuffle=False)

# LSTM
# __init__ is basically a function which will "initialize"/"activate" the properties of the class for a specific object
# self represents that object which will inherit those properties
class simpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(simpleLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
#     def forward(self, x):
#         # 初始化隐藏状态h0, c0为全0向量
#         h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
#         c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

#         # 将输入x和隐藏状态(h0, c0)传入LSTM网络
#         out, _ = self.lstm(x, (h0, c0))
#         # 取最后一个时间步的输出作为LSTM网络的输出
#         out = self.fc(out[:, -1, :])
#         return out

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        # 初始化hidden和memory cell参数
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # forward propagate lstm
        out, (h_n, h_c) = self.lstm(x, (h0, c0))

        # 选取最后一个时刻的输出
        out = self.fc(out[:, -1, :])
        return out

model = simpleLSTM(input_size, hidden_size, num_layers, num_classes)
model.to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr)

# train the model
# 关于reshape(-1)的解释 https://www.zhihu.com/question/52684594
# view()和reshape()区别的解释 https://stackoverflow.com/questions/49643225/whats-the-difference-between-reshape-and-view-in-pytorch



total_step = len(train_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, time_step, input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [1/938], Loss: 2.3088
Epoch [1/10], Step [101/938], Loss: 0.7709
Epoch [1/10], Step [201/938], Loss: 0.3712
Epoch [1/10], Step [301/938], Loss: 0.2808
Epoch [1/10], Step [401/938], Loss: 0.1265
Epoch [1/10], Step [501/938], Loss: 0.1502
Epoch [1/10], Step [601/938], Loss: 0.2009
Epoch [1/10], Step [701/938], Loss: 0.0359
Epoch [1/10], Step [801/938], Loss: 0.0661
Epoch [1/10], Step [901/938], Loss: 0.0940
Epoch [2/10], Step [1/938], Loss: 0.2269
Epoch [2/10], Step [101/938], Loss: 0.0407
Epoch [2/10], Step [201/938], Loss: 0.0941
Epoch [2/10], Step [301/938], Loss: 0.1174
Epoch [2/10], Step [401/938], Loss: 0.0797
Epoch [2/10], Step [501/938], Loss: 0.1740
Epoch [2/10], Step [601/938], Loss: 0.0220
Epoch [2/10], Step [701/938], Loss: 0.0999
Epoch [2/10], Step [801/938], Loss: 0.0529
Epoch [2/10], Step [901/938], Loss: 0.1254
Epoch [3/10], Step [1/938], Loss: 0.1738
Epoch [3/10], Step [101/938], Loss: 0.0906
Epoch [3/10], Step [201/938], Loss: 0.0648
Epoch [3/10], Ste

In [11]:
len(train_loader),len(test_loader)

(938, 157)

In [16]:
correct = 0
total = 0
for images, labels in test_loader:
    images = images.reshape(-1, time_step, input_size).to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.05 %


In [17]:
# Test the model
# https://stackoverflow.com/questions/55627780/evaluating-pytorch-models-with-torch-no-grad-vs-model-eval
# torch.max()用法。https://blog.csdn.net/weixin_43255962/article/details/84402586
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, time_step, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.05 %


In [ ]:
predicted

In [18]:
torch.cuda.empty_cache()